In [115]:
from neo4j import GraphDatabase

In [116]:
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "qwer123456")

driver = GraphDatabase.driver(URI, auth=AUTH)
driver

In [117]:
driver.execute_query("match (n) detach delete n", database_='neo4j') # 删除所有节点和关系

cql_string = """CREATE
(xiaowang_:Person {name: 'xiaowang', bornIn: '湖南', age:34}),
(dc_:Person {name: 'dc', bornIn: '广东', age:41}),
(xiaohong_:Person {name: 'xiaohong', bornIn: '上海', age:51}),
(xiaohei_:Person {name: 'xiaohei', bornIn: '湖南', age:66}),
(dmm_:Person {name: 'dmm', bornIn: '湖南', age:8}),

(dhxy:Movie {title: '大话西游'}),
(bfx:Movie {title: '蝙蝠侠'}),
(hkdg:Movie {title: '黑客帝国'}),

(xiaowang_)-[:like]->(dhxy),
(dc_)-[:dislike]->(bfx),
(xiaohong_)-[:like]->(hkdg),
(xiaohei_)-[:like]->(hkdg),
(dmm_)-[:like]->(hkdg),
(dmm_)-[:dislike]->(bfx)"""
driver.execute_query(cql_string, database_='neo4j')

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x00000212623F5D80>, keys=[])

In [118]:
# 创建一个新节点(数据库中不存在该节点)
driver.execute_query("merge (wangcai:dog {name: 'wangcai'}) return wangcai", database_='neo4j')

EagerResult(records=[<Record wangcai=<Node element_id='11' labels=frozenset({'dog'}) properties={'name': 'wangcai'}>>], summary=<neo4j._work.summary.ResultSummary object at 0x0000021262367970>, keys=['wangcai'])

In [119]:
# 不再创建(数据库中已存在该节点)
driver.execute_query("merge (wangcai:dog {name: 'wangcai'}) return wangcai", database_='neo4j')

EagerResult(records=[<Record wangcai=<Node element_id='11' labels=frozenset({'dog'}) properties={'name': 'wangcai'}>>], summary=<neo4j._work.summary.ResultSummary object at 0x00000212623D8C10>, keys=['wangcai'])

In [120]:
# 仍然创建(数据库中已存在该节点)
driver.execute_query("create (wangcai:dog {name: 'wangcai'}) return wangcai", database_='neo4j')

EagerResult(records=[<Record wangcai=<Node element_id='12' labels=frozenset({'dog'}) properties={'name': 'wangcai'}>>], summary=<neo4j._work.summary.ResultSummary object at 0x00000212623F5E40>, keys=['wangcai'])

In [121]:
# 创建一个新节点(该节点不为数据库中某节点的子集)
# 该节点相比:dog {name: 'wangcai'}包含新属性age
driver.execute_query("merge (wangcai:dog {name: 'wangcai', age: 3}) return wangcai", database_='neo4j')

EagerResult(records=[<Record wangcai=<Node element_id='13' labels=frozenset({'dog'}) properties={'name': 'wangcai', 'age': 3}>>], summary=<neo4j._work.summary.ResultSummary object at 0x00000212623D8F70>, keys=['wangcai'])

In [122]:
# 不再创建(该节点为数据库中某节点的子集)
# 该节点为:Person {name: 'xiaowang', bornIn: '湖南', age:34的子集
driver.execute_query("merge (xiaowang_:Person {name: 'xiaowang', age:34}) return xiaowang_", database_='neo4j')

EagerResult(records=[<Record xiaowang_=<Node element_id='15' labels=frozenset({'Person'}) properties={'bornIn': '湖南', 'name': 'xiaowang', 'age': 34}>>], summary=<neo4j._work.summary.ResultSummary object at 0x00000212623D8430>, keys=['xiaowang_'])

In [123]:
# 不再创建(数据库中已存在该关系)
driver.execute_query("match "
                     "(xiaowang_:Person {name: 'xiaowang'}), "
                     "(hkdg:Movie {title: '黑客帝国'}) "
                     "merge (xiaowang_)-[r:like]->(hkdg)"
                     "return xiaowang_.name, hkdg.title", database_='neo4j')

EagerResult(records=[<Record xiaowang_.name='xiaowang' hkdg.title='黑客帝国'>], summary=<neo4j._work.summary.ResultSummary object at 0x00000212622B0370>, keys=['xiaowang_.name', 'hkdg.title'])

In [124]:
# 创建一条新关系(数据库中不存在该关系)
driver.execute_query("match "
                     "(xiaowang_:Person {name: 'xiaowang'}), "
                     "(dmm_:Person {name: 'dmm'}) "
                     "merge (xiaowang_)-[r:knows]->(dmm_) "
                     "return r", database_='neo4j')

EagerResult(records=[<Record r=<Relationship element_id='17' nodes=(<Node element_id='15' labels=frozenset() properties={}>, <Node element_id='23' labels=frozenset() properties={}>) type='knows' properties={}>>], summary=<neo4j._work.summary.ResultSummary object at 0x00000212623D31C0>, keys=['r'])

In [125]:
driver.close()